In [3]:
import pandas as pd
import first_analysis
#ainda não sei se funciona este import!!!
#import ProFET_master.ProFET.feat_extract.enzyme_descriptors as enzyme_descriptors
import enzyme_descriptors
import kd_transformations
import compound_descriptors
import prepare_to_ML
#import ML_models

In [4]:
#import data to a pandas object from csv file
data_all = pd.read_csv('data/DtcDrugTargetInteractions.csv', sep=',')

/Users/martagomes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3,8,21,25,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_all.shape

(5867203, 32)

In [6]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5867203 entries, 0 to 5867202
Data columns (total 32 columns):
compound_id                          object
standard_inchi_key                   object
compound_name                        object
synonym                              object
target_id                            object
target_pref_name                     object
gene_names                           object
wildtype_or_mutant                   object
mutation_info                        object
pubmed_id                            float64
standard_type                        object
standard_relation                    object
standard_value                       float64
standard_units                       object
ep_action_mode                       object
assay_format                         object
assaytype                            object
assay_subtype                        object
inhibitor_type                       object
detection_tech                       object
assay

In [7]:
import numpy as np
indices = np.random.permutation(5867203)
#print(indices)

In [16]:
print(type(indices[0:500]))

<class 'numpy.ndarray'>


In [20]:
index = list(indices[0:500])

In [21]:
type(index)

list

In [22]:
len(index)

500

In [23]:
data = data_all.iloc[index,:]

In [24]:
data.shape

(500, 32)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1627136 to 2879323
Data columns (total 32 columns):
compound_id                          498 non-null object
standard_inchi_key                   493 non-null object
compound_name                        71 non-null object
synonym                              0 non-null object
target_id                            499 non-null object
target_pref_name                     500 non-null object
gene_names                           388 non-null object
wildtype_or_mutant                   10 non-null object
mutation_info                        0 non-null object
pubmed_id                            177 non-null float64
standard_type                        500 non-null object
standard_relation                    303 non-null object
standard_value                       468 non-null float64
standard_units                       461 non-null object
ep_action_mode                       14 non-null object
assay_format                       

In [26]:
first_analysis.look_at_data(data)

Data head:

           compound_id           standard_inchi_key          compound_name  \
1627136  CHEMBL1462008  HURVNUBCCDEKRW-ZRDIBKRKSA-N                    NaN   
1232359  CHEMBL1382988  GDPMPIOBOVLEFC-UHFFFAOYSA-N                    NaN   
4495953   CHEMBL221753  UREZNYTWGJKWBI-UHFFFAOYSA-M  BENZETHONIUM CHLORIDE   
164163        CHEMBL30  AQIXAKUUQRKLND-UHFFFAOYSA-N             CIMETIDINE   
5617646  CHEMBL1535676  ZIPNPHODCJOAKC-UHFFFAOYSA-N                    NaN   

        synonym target_id                               target_pref_name  \
1627136     NaN    P04637                     CELLULAR TUMOR ANTIGEN P53   
1232359     NaN    Q16236    NUCLEAR FACTOR ERYTHROID 2-RELATED FACTOR 2   
4495953     NaN    P25101                       ENDOTHELIN RECEPTOR ET-A   
164163      NaN    P41145                          KAPPA OPIOID RECEPTOR   
5617646     NaN    Q96QE3  ATPASE FAMILY AAA DOMAIN-CONTAINING PROTEIN 5   

        gene_names wildtype_or_mutant mutation_info  pubmed_id

In [27]:
data = data.dropna(subset=['standard_value'])

In [28]:
print(data.isnull().sum())

compound_id                            2
standard_inchi_key                     7
compound_name                        416
synonym                              468
target_id                              1
target_pref_name                       0
gene_names                           107
wildtype_or_mutant                   459
mutation_info                        468
pubmed_id                            304
standard_type                          0
standard_relation                    165
standard_value                         0
standard_units                         7
ep_action_mode                       455
assay_format                         415
assaytype                            455
assay_subtype                        455
inhibitor_type                       463
detection_tech                       456
assay_cell_line                      444
compound_concentration_value         457
compound_concentration_value_unit    457
substrate_type                       458
substrate_relati

In [29]:
#By default keep the lines with the following values in standard_type : ["PKD", "LOGKD", "LOG KD", "LOG 1/KD", "Kd", "KD\'", "-LOG KD", "KD"]
print(data.shape)
data = kd_transformations.filter_standard_type(data, keep=True, list=['KD','Kd'])
print(data.shape)

(468, 32)
(5, 32)


In [30]:
#Confirm that is only kd values in NM units 
print(data.compound_id.groupby(data.standard_type).count())
data = data[data.standard_units == 'NM']
print(data.standard_type.groupby(data.standard_units).count())

standard_type
KD    5
Name: compound_id, dtype: int64
standard_units
NM    5
Name: standard_type, dtype: int64


In [31]:
print(data.standard_value)
#Conversion to pkd values in the correct unites
data = kd_transformations.conversion(data)
#Check if is everything ok with standard values
print(data.isnull().sum())
data.standard_value

2818208       12.59
5111428    10000.00
4406545    10000.00
3410011    10000.00
4494       10000.00
Name: standard_value, dtype: float64
compound_id                          0
standard_inchi_key                   0
compound_name                        1
synonym                              5
target_id                            0
target_pref_name                     0
gene_names                           1
wildtype_or_mutant                   4
mutation_info                        5
pubmed_id                            0
standard_type                        0
standard_relation                    0
standard_value                       0
standard_units                       0
ep_action_mode                       4
assay_format                         3
assaytype                            4
assay_subtype                        4
inhibitor_type                       5
detection_tech                       4
assay_cell_line                      5
compound_concentration_value         4
compo

2818208    7.899974
5111428    5.000000
4406545    5.000000
3410011    5.000000
4494       5.000000
Name: standard_value, dtype: float64

In [32]:
#COMPLETE COMPOUND ID WITH COMPOUND NAME
data = first_analysis.complete_cols(data, 'compound_id', 'compound_name')
#see if there were any changes
print('Null values for column: \n')
print(data.isnull().sum())
print('\n')


#COMPLETE COUMPOUND ID WITH STANDARD_INCHI_KEY
data = first_analysis.complete_cols(data, 'compound_id', 'standard_inchi_key')
#see if there were any changes
print('Null values for column: \n')
print(data.isnull().sum())
print('\n')


#COMPLETE STANDARD_INCHI_KEY WITH COMPOUND ID
data = first_analysis.complete_cols(data, 'standard_inchi_key', 'compound_id')
#see if there were any changes
print('Null values for column: \n')
print(data.isnull().sum())
print('\n')

Null values for column: 

compound_id                          0
standard_inchi_key                   0
compound_name                        1
synonym                              5
target_id                            0
target_pref_name                     0
gene_names                           1
wildtype_or_mutant                   4
mutation_info                        5
pubmed_id                            0
standard_type                        0
standard_relation                    0
standard_value                       0
standard_units                       0
ep_action_mode                       4
assay_format                         3
assaytype                            4
assay_subtype                        4
inhibitor_type                       5
detection_tech                       4
assay_cell_line                      5
compound_concentration_value         4
compound_concentration_value_unit    4
substrate_type                       4
substrate_relation                   5

In [33]:
data.shape

(5, 32)

In [34]:
#GET COMPOUND DESCRIPTORS (lines with nan compound_id values are deleted)
data = compound_descriptors.get_compound_descriptors(data)
print(data.isnull().sum())
print(data.shape)

Morgan
MACCS
MolLogP
compound_id                          0
standard_inchi_key                   0
compound_name                        1
target_id                            0
target_pref_name                     0
gene_names                           1
wildtype_or_mutant                   4
pubmed_id                            0
standard_type                        0
standard_relation                    0
standard_value                       0
standard_units                       0
ep_action_mode                       4
assay_format                         3
assaytype                            4
assay_subtype                        4
detection_tech                       4
compound_concentration_value         4
compound_concentration_value_unit    4
substrate_type                       4
assay_description                    0
title                                0
journal                              0
doc_type                             0
smiles                               0
stan

In [17]:
data.to_csv('data_only_compound_descriptord.csv')

In [2]:
data=pd.read_csv('data_only_compound_descriptord.csv',index_col = 0)

/Users/martagomes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4,8,9,15,16,17,18,19,20,21,22,23,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
data.head()

,compound_id,standard_inchi_key,compound_name,target_id,target_pref_name,gene_names,wildtype_or_mutant,pubmed_id,standard_type,standard_relation,...,MACCS_158,MACCS_159,MACCS_160,MACCS_161,MACCS_162,MACCS_163,MACCS_164,MACCS_165,MACCS_166,MolLogP
0,CHEMBL69901,MVPWJPFOERRNKK-UHFFFAOYSA-N,NaN,Q9JI35,HISTAMINE H3 RECEPTOR,NaN,NaN,9873505.0,KD,=,...,1,1,0,1,1,1,1,1,0,2.3643
1,CHEMBL939,XGALLCVXEZPNRQ-UHFFFAOYSA-N,GEFITINIB,P78362,SERINE/THREONINE-PROTEIN KINASE SRPK2,SRPK2,wild_type,22037378.0,KD,>,...,1,1,1,1,1,1,1,1,0,4.2756
2,CHEMBL24828,UHTHHESEBZOYNR-UHFFFAOYSA-N,VANDETANIB,Q9BRS2,SERINE/THREONINE-PROTEIN KINASE RIO1,RIOK1,NaN,18183025.0,KD,>,...,1,1,1,1,1,1,1,1,0,5.0042
3,CHEMBL522892,PIQCTGMSNWUMAF-UHFFFAOYSA-N,DOVITINIB,P31749,SERINE/THREONINE-PROTEIN KINASE AKT,AKT1,NaN,18183025.0,KD,>,...,1,0,1,1,1,1,1,1,0,2.5445
4,CHEMBL553,AAKJLRGGTJKAMG-UHFFFAOYSA-N,ERLOTINIB,P23458,TYROSINE-PROTEIN KINASE JAK1,JAK1,NaN,22037378.0,KD,>,...,1,1,1,1,1,1,1,1,0,3.4051


In [36]:
#To test ProFet and define chaves values
seq = 'MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL'
dic = enzyme_descriptors.Get_Protein_Feat(seq, SeqReducedAlph='ofer14', ReducedK=2, GetSimpleFeatSet=True, GetExtraScaleFeatSet=True,
                       aaParamScaleWindow=7, ExtraScaleWindow=17, GetSubSeqSegs=True, SubSeqSegs=3,
                       GetTriLetterGroupAlphKFreq=True, TriLetterGroupAlphK=5, GetSeqReducedGroups=True,
                       SeqReducedGroups='ofer_w8', GetSeqReducedAlph=True, GetCTDFeatSet=True,
                       GetPTMFeatSet=True, GetDBCleavageFeatSet=True, split_N=False, split_C=False,
                       N_TAIL_REGION=30, C_TAIL_REGION=30)

print(dic)

{'AA: M Frequency': 1.353, 'AA: K Frequency': 5.099, 'AA: F Frequency': 3.434, 'AA: D Frequency': 4.683, 'AA: S Frequency': 8.325, 'AA: R Frequency': 3.33, 'AA: E Frequency': 4.475, 'AA: Q Frequency': 11.655, 'AA: G Frequency': 6.764, 'AA: L Frequency': 8.741, 'AA: T Frequency': 6.452, 'AA: Y Frequency': 1.561, 'AA: I Frequency': 4.162, 'AA: V Frequency': 5.307, 'AA: A Frequency': 8.221, 'AA: N Frequency': 3.434, 'AA: C Frequency': 1.353, 'AA: H Frequency': 1.561, 'AA: W Frequency': 0.312, 'AA: P Frequency': 9.781, 'M Entropy': 6.207952902632658, 'K Entropy': 4.293682776658542, 'F Entropy': 4.863998501415297, 'D Entropy': 4.416539524444076, 'S Entropy': 3.586464525886388, 'R Entropy': 4.90839262077375, 'E Entropy': 4.482127866071653, 'Q Entropy': 3.101037698716146, 'G Entropy': 3.886024807745296, 'L Entropy': 3.51607519799499, 'T Entropy': 3.9541963103868754, 'Y Entropy': 6.001502025165232, 'I Entropy': 4.586464525886388, 'V Entropy': 4.235967278802255, 'A Entropy': 3.6046118725966476,

In [37]:
#GET ENZYME SEQUENCES
#remove rows without target_id
data = data[~data.target_id.isnull()]
data = enzyme_descriptors.add_sequence_col(data, 'target_id')
data.shape

(5, 1220)

In [38]:
print(data.sequence.isnull().sum())

0


In [39]:
#GET ENZYME FEATURES FROM SEQUENCES
data = enzyme_descriptors.add_ProFet_features(data)
data.isnull().sum()

Get_Protein_Feat done :D !
Get_Protein_Feat done :D !
Get_Protein_Feat done :D !
Get_Protein_Feat done :D !
Get_Protein_Feat done :D !
Get_Protein_Feat done :D !


compound_id                          0
standard_inchi_key                   0
compound_name                        1
target_id                            0
target_pref_name                     0
gene_names                           1
wildtype_or_mutant                   4
pubmed_id                            0
standard_type                        0
standard_relation                    0
standard_value                       0
standard_units                       0
ep_action_mode                       4
assay_format                         3
assaytype                            4
assay_subtype                        4
detection_tech                       4
compound_concentration_value         4
compound_concentration_value_unit    4
substrate_type                       4
assay_description                    0
title                                0
journal                              0
doc_type                             0
smiles                               0
standard_inchi           

In [40]:
data.shape

(5, 2393)

In [9]:
data1 = data[data.target_id != 'P08485']
data1.isnull().sum()

compound_id                              0
standard_inchi_key                     895
compound_name                        20916
synonym                              70631
target_id                                0
target_pref_name                         0
gene_names                            9063
wildtype_or_mutant                   44639
mutation_info                        70283
pubmed_id                             3115
standard_type                            0
standard_relation                       13
standard_value                           0
standard_units                           0
ep_action_mode                       44709
assay_format                         41275
assaytype                            44706
assay_subtype                        44714
inhibitor_type                       71722
detection_tech                       44708
assay_cell_line                      70379
compound_concentration_value         44742
compound_concentration_value_unit    44741
substrate_t

In [10]:
data1.to_csv('data_all_descriptors.csv')